In [4]:
import numpy as np
import pandas as pd

import pitched_recommend

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import cluster, metrics

import matplotlib.pyplot as plt
import seaborn as sns

import pickle

sns.set(style="darkgrid", palette="Set3")

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
with open('/Users/meshchd/Downloads/recommender_model_model_20190504.pkl', 'rb') as f:
    model = pickle.load(f)

In [8]:
df = pd.DataFrame(data=model.playlist_vecs, index=model.playlists, 
                  columns=['factor'+str(x) for x in range(1,model.playlist_vecs.shape[1]+1)])
df.head()

,factor1,factor2,factor3,factor4,factor5,factor6,factor7,factor8,factor9,factor10,...,factor191,factor192,factor193,factor194,factor195,factor196,factor197,factor198,factor199,factor200
spotify:playlist:0000mSEZofZjMa5x6ooFMT,0.381671,4.340792,-1.436632,2.492162,-0.077138,2.750425,0.692091,-2.326943,-0.939862,-2.059456,...,-2.235541,-0.296701,-1.148543,-1.758255,1.660909,5.379026,-0.291825,-1.676979,2.763846,1.001892
spotify:playlist:0003EPZgOqan8RIyLg3zIE,1.095493,-1.625466,4.475720,0.537307,-0.210129,-0.762072,-2.743342,0.807074,7.351343,-3.486826,...,3.537256,-2.377482,-2.372458,-4.638627,0.412767,0.448122,-1.298666,-2.450981,5.587291,5.880754
spotify:playlist:000H8pt2u6SWMvNtTksPbC,0.550884,-0.166518,0.682681,-1.251670,-2.490389,-0.329858,0.481624,5.699526,1.322156,1.290797,...,0.271886,0.099851,2.013634,0.632457,2.816938,0.240414,-2.903885,5.586271,-5.823388,4.404129
spotify:playlist:000IDLrAdMwaTycyWNqphh,0.563976,-0.305774,1.505696,-1.138674,0.289151,0.143725,-0.369537,-0.770468,-0.679030,0.513139,...,-0.160115,-0.102442,1.711999,0.837739,-0.324243,0.518000,0.144128,-0.704987,-0.601334,0.619642
spotify:playlist:000T04pNN05jXDimzYbl4b,-0.759061,4.314501,2.325355,-0.824923,-2.003628,-0.356305,4.351025,-0.281910,2.411831,-1.049518,...,-0.551718,2.670736,-1.979285,2.621447,-1.966242,0.157424,-0.549873,0.273648,1.155447,-1.852015


In [9]:
df.describe()

,factor1,factor2,factor3,factor4,factor5,factor6,factor7,factor8,factor9,factor10,...,factor191,factor192,factor193,factor194,factor195,factor196,factor197,factor198,factor199,factor200
count,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,...,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000,406692.000000
mean,0.165646,0.116947,0.332807,0.338597,-0.477668,0.251461,0.370317,0.298048,0.461898,0.145712,...,0.041779,0.173160,0.243934,0.107502,0.380591,0.119329,0.093517,0.209750,-0.037036,0.438861
std,2.270700,2.231738,2.329095,2.382953,2.256947,2.333773,2.203802,2.310757,2.406285,2.126753,...,2.221420,2.163310,2.193532,2.338506,2.245787,2.306930,2.308897,2.252246,2.188720,2.394541
min,-18.923000,-16.023695,-18.038120,-19.094032,-13.223364,-16.092640,-13.619540,-14.302153,-19.710732,-13.895269,...,-14.753638,-15.495889,-16.017120,-18.372465,-16.567102,-16.271761,-12.615186,-14.396374,-15.910635,-19.507126
25%,-1.151003,-1.162591,-1.120806,-1.057571,-1.852280,-1.118876,-0.902374,-1.102748,-0.920255,-1.009961,...,-1.251449,-1.085911,-0.934026,-1.219115,-0.840363,-1.302939,-1.272481,-1.167900,-1.357419,-1.041806
50%,0.254910,0.151441,0.257424,0.356830,-0.515890,0.286785,0.351670,0.218330,0.432046,0.173758,...,0.132940,0.246988,0.292522,0.132236,0.447336,0.065279,-0.078418,0.169964,-0.081600,0.473606
75%,1.609796,1.414963,1.752454,1.752530,0.824961,1.682808,1.641605,1.644416,1.885753,1.347393,...,1.418076,1.495290,1.474720,1.546581,1.651013,1.473901,1.349050,1.526556,1.249564,1.873814
max,17.030474,13.823609,18.097679,17.507784,15.952332,17.063221,16.925486,17.061020,15.312299,16.182711,...,15.343296,13.831488,16.158329,21.552904,17.085917,16.593182,17.325550,17.982298,16.719412,20.540712
